# Multi-City Green Space Detection
## Training Random Forest with WorldCover 2021 as Ground Truth

Training cities: 11 cities for robust model generalisation

- Uses WorldCover 2021 as ground truth (green classes: tree cover, shrubland, grassland, mangroves)
- Multi-temporal Sentinel-2 data (April, August, November)
- 21 bands: 4 spectral + 3 vegetation indices per month
- Cross-city training for better generalisation

## 1. Import Libraries

In [ ]:
%pip install numpy rasterio geopandas scikit-learn matplotlib seaborn tqdm

import json
import os
import glob
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
from pathlib import Path
import geopandas as gpd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported")

## 2. Configuration

In [4]:
# Base paths - using relative paths from project root
# Run notebooks from the project root directory: python -m jupyter notebook
import os

# Find project root (go up from notebooks/training/)
NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, "..", ".."))

# If running from project root directly
if os.path.exists("data") and os.path.exists("models"):
    PROJECT_ROOT = os.getcwd()
# If running from notebooks/training/
elif os.path.exists("../../data") and os.path.exists("../../models"):
    PROJECT_ROOT = os.path.abspath("../..")
# Fallback
else:
    PROJECT_ROOT = os.getcwd()
    print(f"Warning: Could not detect project root. Using: {PROJECT_ROOT}")

# Derived paths
DATA_PATH = os.path.join(PROJECT_ROOT, "data")
MODELS_PATH = os.path.join(PROJECT_ROOT, "models")
GEOJSON_FOLDER = os.path.join(DATA_PATH, "aois")

# Output folder
OUTPUT_FOLDER = os.path.join(PROJECT_ROOT, "outputs", "multi_city_training")
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
os.makedirs(MODELS_PATH, exist_ok=True)

# Create timestamped run folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_FOLDER = os.path.join(OUTPUT_FOLDER, f"run_{timestamp}")
os.makedirs(RUN_FOLDER, exist_ok=True)

# WorldCover green classes
GREEN_CLASSES = [10, 20, 30, 95]  # Tree, Shrub, Grass, Mangroves

# Define cities with their specific file locations
# All data now in unified data/ folder structure
CITY_FILES = {
    "Amsterdam": {
        "stack": os.path.join(DATA_PATH, "sentinel_stacks", "Amsterdam_MultiMonth_stack.tif"),
        "worldcover": os.path.join(DATA_PATH, "worldcover", "Amsterdam_WorldCover_2021.tif"),
        "geojson": os.path.join(GEOJSON_FOLDER, "Amsterdam.geojson"),
    },
    "Auckland": {
        "stack": os.path.join(DATA_PATH, "sentinel_stacks", "Auckland_MultiMonth_stack.tif"),
        "worldcover": os.path.join(DATA_PATH, "worldcover", "Auckland_WorldCover_2021.tif"),
        "geojson": os.path.join(GEOJSON_FOLDER, "Auckland.geojson"),
    },
    "Barcelona": {
        "stack": os.path.join(DATA_PATH, "sentinel_stacks", "Barcelona_MultiMonth_stack.tif"),
        "worldcover": os.path.join(DATA_PATH, "worldcover", "Barcelona_WorldCover_2021.tif"),
        "geojson": os.path.join(GEOJSON_FOLDER, "Barcelona.geojson"),
    },
    "Sydney": {
        "stack": os.path.join(DATA_PATH, "sentinel_stacks", "Sydney_MultiMonth_stack.tif"),
        "worldcover": os.path.join(DATA_PATH, "worldcover", "Sydney_WorldCover_2021.tif"),
        "geojson": os.path.join(GEOJSON_FOLDER, "Sydney.geojson"),
    },
    "Toronto": {
        "stack": os.path.join(DATA_PATH, "sentinel_stacks", "Toronto_MultiMonth_stack.tif"),
        "worldcover": os.path.join(DATA_PATH, "worldcover", "Toronto_WorldCover_2021.tif"),
        "geojson": os.path.join(GEOJSON_FOLDER, "Toronto.geojson"),
    },
    "Vienna": {
        "stack": os.path.join(DATA_PATH, "sentinel_stacks", "Wien_MultiMonth_stack.tif"),
        "worldcover": os.path.join(DATA_PATH, "worldcover", "Vienna_WorldCover_2021.tif"),
        "geojson": os.path.join(GEOJSON_FOLDER, "Vienna.geojson"),
    },
    "London": {
        "stack": os.path.join(DATA_PATH, "sentinel_stacks", "London_MultiMonth_stack.tif"),
        "worldcover": os.path.join(DATA_PATH, "worldcover", "London_WorldCover_2021.tif"),
        "geojson": os.path.join(GEOJSON_FOLDER, "London.geojson"),
    },
    "Melbourne": {
        "stack": os.path.join(DATA_PATH, "sentinel_stacks", "Melbourne_MultiMonth_stack.tif"),
        "worldcover": os.path.join(DATA_PATH, "worldcover", "Melbourne_WorldCover_2021.tif"),
        "geojson": os.path.join(GEOJSON_FOLDER, "Melbourne.geojson"),
    },
    "Paris": {
        "stack": os.path.join(DATA_PATH, "sentinel_stacks", "Paris_MultiMonth_stack.tif"),
        "worldcover": os.path.join(DATA_PATH, "worldcover", "Paris_WorldCover_2021.tif"),
        "geojson": os.path.join(GEOJSON_FOLDER, "Paris.geojson"),
    },
    "San_Francisco": {
        "stack": os.path.join(DATA_PATH, "sentinel_stacks", "San_Francisco_MultiMonth_stack.tif"),
        "worldcover": os.path.join(DATA_PATH, "worldcover", "San_Francisco_WorldCover_2021.tif"),
        "geojson": os.path.join(GEOJSON_FOLDER, "San_Francisco.geojson"),
    },
    "Seattle": {
        "stack": os.path.join(DATA_PATH, "sentinel_stacks", "Seattle_MultiMonth_stack.tif"),
        "worldcover": os.path.join(DATA_PATH, "worldcover", "Seattle_WorldCover_2021.tif"),
        "geojson": os.path.join(GEOJSON_FOLDER, "Seattle.geojson"),
    },
}

print("Configuration loaded")
print(f"  Project root: {PROJECT_ROOT}")
print(f"  Data path: {DATA_PATH}")
print(f"  Models path: {MODELS_PATH}")
print(f"  Output folder: {RUN_FOLDER}")
print(f"  Target cities: {len(CITY_FILES)}")
for city in CITY_FILES:
    print(f"    - {city}")

Configuration loaded
  Project root: /Users/tyomachka/Desktop/WU/Data_Lab.TMP/rep.infrared.city
  Data path: /Users/tyomachka/Desktop/WU/Data_Lab.TMP/rep.infrared.city/data
  Models path: /Users/tyomachka/Desktop/WU/Data_Lab.TMP/rep.infrared.city/models
  Output folder: /Users/tyomachka/Desktop/WU/Data_Lab.TMP/rep.infrared.city/outputs/multi_city_training/run_20260127_231351
  Target cities: 11
    - Amsterdam
    - Auckland
    - Barcelona
    - Sydney
    - Toronto
    - Vienna
    - London
    - Melbourne
    - Paris
    - San_Francisco
    - Seattle


## 3. Discover Available Cities
### Find all cities with Multi-Month stacks

In [ ]:
print("=" * 70)
print("DISCOVERING AVAILABLE CITIES")
print("=" * 70)

print(f"\nChecking {len(CITY_FILES)} configured cities...")

cities_data = []
missing_cities = []

for city_name, paths in CITY_FILES.items():
    stack_file = paths["stack"]
    worldcover_file = paths["worldcover"]
    geojson_file = paths["geojson"]
    
    has_stack = os.path.exists(stack_file)
    has_worldcover = os.path.exists(worldcover_file)
    has_geojson = os.path.exists(geojson_file)
    
    status_stack = "Y" if has_stack else "N"
    status_geojson = "Y" if has_geojson else "N"
    status_worldcover = "Y" if has_worldcover else "N"
    
    print(f"  {city_name:15s} - Stack: {status_stack}  GeoJSON: {status_geojson}  WorldCover: {status_worldcover}")
    
    if has_stack and has_geojson and has_worldcover:
        cities_data.append({
            "name": city_name,
            "stack_file": stack_file,
            "geojson_file": geojson_file,
            "worldcover_file": worldcover_file
        })
    else:
        missing = []
        if not has_stack: missing.append("Stack")
        if not has_geojson: missing.append("GeoJSON")
        if not has_worldcover: missing.append("WorldCover")
        missing_cities.append(f"{city_name} (missing: {', '.join(missing)})")

complete_cities = cities_data

print(f"\n{'=' * 70}")
print(f"Cities with complete data: {len(complete_cities)}/{len(CITY_FILES)}")
print(f"{'=' * 70}")

if missing_cities:
    print(f"\nSkipped cities:")
    for city in missing_cities:
        print(f"  - {city}")

if len(complete_cities) == 0:
    raise ValueError("No cities with complete data found! Check your file paths.")

print(f"\nReady to train with {len(complete_cities)} cities:")
for city in complete_cities:
    print(f"  - {city['name']}")

## 4. Load and Process All Cities
### Load Sentinel-2 stacks and create WorldCover labels for each city

In [ ]:
print("\n" + "=" * 70)
print("LOADING AND PROCESSING ALL CITIES")
print("=" * 70)

EXPECTED_BANDS = None

# cap per class per city to keep memory manageable and balance the dataset
MAX_SAMPLES_PER_CLASS_PER_CITY = 100000

all_X = []
all_y = []
city_info = []
skipped_cities = []

for city_data in tqdm(complete_cities, desc="Processing cities"):
    city_name = city_data["name"]
    stack_file = city_data["stack_file"]
    worldcover_file = city_data["worldcover_file"]
    
    print(f"\n{'=' * 70}")
    print(f"Processing: {city_name}")
    print(f"{'=' * 70}")
    
    try:
        with rasterio.open(stack_file) as src:
            X_stack = src.read()
            stack_transform = src.transform
            stack_shape = (src.height, src.width)
            stack_crs = src.crs
        
        n_bands = X_stack.shape[0]
        print(f"  Loaded Sentinel-2 stack: {X_stack.shape} ({n_bands} bands)")
        
        if EXPECTED_BANDS is None:
            EXPECTED_BANDS = n_bands
            print(f"  Setting expected bands to {EXPECTED_BANDS}")
        elif n_bands != EXPECTED_BANDS:
            print(f"  SKIPPING: Expected {EXPECTED_BANDS} bands, but found {n_bands}")
            skipped_cities.append({"name": city_name, "reason": f"Band mismatch: {n_bands} vs {EXPECTED_BANDS}"})
            continue
        
        # reproject WorldCover to match Sentinel-2 grid
        with rasterio.open(worldcover_file) as src:
            worldcover_data = np.empty(stack_shape, dtype=np.uint8)
            reproject(
                source=rasterio.band(src, 1),
                destination=worldcover_data,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=stack_transform,
                dst_crs=stack_crs,
                resampling=Resampling.nearest
            )
        
        # binary green / non-green
        labels = np.isin(worldcover_data, GREEN_CLASSES).astype(np.uint8)
        
        green_percentage = 100 * labels.sum() / labels.size
        print(f"  WorldCover labels: {labels.shape} ({green_percentage:.2f}% green)")
        
        n_pixels = X_stack.shape[1] * X_stack.shape[2]
        X = X_stack.reshape(n_bands, -1).T
        y = labels.flatten()
        
        valid_mask = ~np.isnan(X).any(axis=1)
        X_clean = X[valid_mask]
        y_clean = y[valid_mask]
        
        print(f"  Valid samples: {len(X_clean):,} ({100 * len(X_clean) / n_pixels:.1f}% of pixels)")
        
        # balanced sampling: equal green and non-green
        green_indices = np.where(y_clean == 1)[0]
        nongreen_indices = np.where(y_clean == 0)[0]
        
        n_samples_per_class = min(len(green_indices), len(nongreen_indices), MAX_SAMPLES_PER_CLASS_PER_CITY)
        
        np.random.seed(42)
        sampled_green = np.random.choice(green_indices, n_samples_per_class, replace=False)
        sampled_nongreen = np.random.choice(nongreen_indices, n_samples_per_class, replace=False)
        
        sampled_indices = np.concatenate([sampled_green, sampled_nongreen])
        np.random.shuffle(sampled_indices)
        
        X_sampled = X_clean[sampled_indices]
        y_sampled = y_clean[sampled_indices]
        
        print(f"  Balanced sampling: {len(X_sampled):,} samples ({n_samples_per_class:,} per class)")
        print(f"    Green: {np.sum(y_sampled == 1):,} ({100 * np.sum(y_sampled == 1) / len(y_sampled):.1f}%)")
        print(f"    Non-green: {np.sum(y_sampled == 0):,} ({100 * np.sum(y_sampled == 0) / len(y_sampled):.1f}%)")
        
        all_X.append(X_sampled)
        all_y.append(y_sampled)
        city_info.extend([city_name] * len(X_sampled))
        
    except Exception as e:
        print(f"  Error processing {city_name}: {e}")
        skipped_cities.append({"name": city_name, "reason": str(e)})
        continue

print(f"\n{'=' * 70}")
print("DATA AGGREGATION")
print(f"{'=' * 70}")

if skipped_cities:
    print(f"\nSkipped {len(skipped_cities)} cities:")
    for city in skipped_cities:
        print(f"  - {city['name']}: {city['reason']}")

if len(all_X) == 0:
    raise ValueError("No valid city data loaded! Check that all stacks have the same number of bands.")

X_combined = np.vstack(all_X)
y_combined = np.hstack(all_y)
city_info = np.array(city_info)

print(f"\nCombined dataset (balanced):")
print(f"  Cities included: {len(all_X)}")
print(f"  Total samples: {len(X_combined):,}")
print(f"  Features (bands): {X_combined.shape[1]}")
print(f"  Green: {np.sum(y_combined == 1):,} ({100 * np.sum(y_combined == 1) / len(y_combined):.1f}%)")
print(f"  Non-green: {np.sum(y_combined == 0):,} ({100 * np.sum(y_combined == 0) / len(y_combined):.1f}%)")

print(f"\nSamples per city:")
for city_name in np.unique(city_info):
    city_samples = np.sum(city_info == city_name)
    print(f"  {city_name:20s}: {city_samples:>10,}")

print(f"{'=' * 70}")

## 5. Train-Test Split
### Split data for training and validation

In [ ]:
print("\n" + "=" * 70)
print("TRAIN-TEST SPLIT & FEATURE NORMALIZATION")
print("=" * 70)

X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y_combined, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_combined
)

print(f"\nDataset split:")
print(f"  Training samples: {len(X_train):,}")
print(f"    Green: {np.sum(y_train == 1):,} ({100 * np.sum(y_train == 1) / len(y_train):.1f}%)")
print(f"    Non-green: {np.sum(y_train == 0):,} ({100 * np.sum(y_train == 0) / len(y_train):.1f}%)")
print(f"\n  Testing samples: {len(X_test):,}")
print(f"    Green: {np.sum(y_test == 1):,} ({100 * np.sum(y_test == 1) / len(y_test):.1f}%)")
print(f"    Non-green: {np.sum(y_test == 0):,} ({100 * np.sum(y_test == 0) / len(y_test):.1f}%)")

print(f"\nApplying StandardScaler...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"  Features normalized")
print(f"    Mean (train): {X_train_scaled.mean():.4f}")
print(f"    Std (train): {X_train_scaled.std():.4f}")

print(f"{'=' * 70}")

## 6. Train Random Forest Model
### Train on multi-city dataset

In [ ]:
print("\n" + "=" * 70)
print("TRAINING RANDOM FOREST MODEL")
print("=" * 70)

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    class_weight=None,  # data already balanced via sampling
    random_state=42,
    n_jobs=-1,
    verbose=1
)

print(f"\nRandom Forest parameters:")
print(f"  n_estimators: {rf.n_estimators}")
print(f"  max_depth: {rf.max_depth}")
print(f"  min_samples_split: {rf.min_samples_split}")
print(f"  min_samples_leaf: {rf.min_samples_leaf}")
print(f"  max_features: {rf.max_features}")
print(f"  class_weight: {rf.class_weight}")

print(f"\nTraining on {len(complete_cities)} cities, {len(X_train_scaled):,} samples...")

rf.fit(X_train_scaled, y_train)

print(f"\nModel trained")
print(f"{'=' * 70}")

## 7. Evaluate Model Performance

In [ ]:
print("\n" + "=" * 70)
print("MODEL EVALUATION")
print("=" * 70)

y_pred = rf.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print(f"\nPerformance ({len(complete_cities)} cities):")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")

cm = confusion_matrix(y_test, y_pred)
print(f"\nConfusion Matrix:")
print(f"                 Predicted")
print(f"               Non-Green  Green")
print(f"Actual Non-Green  {cm[0,0]:>8,}  {cm[0,1]:>8,}")
print(f"       Green      {cm[1,0]:>8,}  {cm[1,1]:>8,}")

total_actual_green = cm[1,0] + cm[1,1]
total_actual_nongreen = cm[0,0] + cm[0,1]
green_detected = 100 * cm[1,1] / total_actual_green if total_actual_green > 0 else 0
green_missed = 100 * cm[1,0] / total_actual_green if total_actual_green > 0 else 0

print(f"\nGreen detection:")
print(f"  Correctly detected: {green_detected:.1f}%")
print(f"  Missed (false negative): {green_missed:.1f}%")

metrics = {
    "model": "RandomForest_Improved",
    "ground_truth": "WorldCover_2021",
    "training_cities": [city['name'] for city in complete_cities],
    "n_cities": len(complete_cities),
    "total_training_samples": int(len(X_train)),
    "total_testing_samples": int(len(X_test)),
    "accuracy": float(accuracy),
    "precision": float(precision),
    "recall": float(recall),
    "f1_score": float(f1),
    "confusion_matrix": cm.tolist(),
    "improvements": [
        "Balanced sampling (equal green/non-green per city)",
        "Feature normalization (StandardScaler)",
        "Increased n_estimators (200)",
        "Removed max_depth limit",
        "Reduced min_samples_split (10)",
        "Reduced min_samples_leaf (5)",
        "Added class_weight='balanced'"
    ]
}

with open(os.path.join(RUN_FOLDER, "metrics.json"), "w") as f:
    json.dump(metrics, f, indent=2)

print(f"\nMetrics saved to: {RUN_FOLDER}/metrics.json")
print(f"{'=' * 70}")

## 8. Visualize Confusion Matrix

In [ ]:
# plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-Green', 'Green'],
            yticklabels=['Non-Green', 'Green'],
            cbar_kws={'label': 'Count'})
plt.title(f'Confusion Matrix - Multi-City Random Forest\n(Trained on {len(complete_cities)} cities)', 
          fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(RUN_FOLDER, 'confusion_matrix.png'), dpi=300, bbox_inches='tight')
plt.show()

print("Confusion matrix saved")

## 9. Feature Importance Analysis

In [ ]:
# feature importances
importances = rf.feature_importances_
n_features = len(importances)

if n_features == 21:
    band_names = [
        'B02-Apr', 'B03-Apr', 'B04-Apr', 'B08-Apr', 'NDVI-Apr', 'EVI-Apr', 'SAVI-Apr',
        'B02-Aug', 'B03-Aug', 'B04-Aug', 'B08-Aug', 'NDVI-Aug', 'EVI-Aug', 'SAVI-Aug',
        'B02-Nov', 'B03-Nov', 'B04-Nov', 'B08-Nov', 'NDVI-Nov', 'EVI-Nov', 'SAVI-Nov'
    ]
elif n_features == 12:
    band_names = [
        'B02-Apr', 'B03-Apr', 'B04-Apr', 'B08-Apr',
        'B02-Aug', 'B03-Aug', 'B04-Aug', 'B08-Aug',
        'B02-Nov', 'B03-Nov', 'B04-Nov', 'B08-Nov'
    ]
else:
    band_names = [f'Band_{i+1}' for i in range(n_features)]
    print(f"Using generic band names for {n_features} bands")

indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, max(8, n_features * 0.4)))
plt.barh(range(len(importances)), importances[indices], color='steelblue')
plt.yticks(range(len(importances)), [band_names[i] for i in indices])
plt.xlabel('Feature Importance', fontsize=12)
plt.title(f'Random Forest Feature Importance\n({len(all_X)} cities, {n_features} bands)', 
          fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(RUN_FOLDER, 'feature_importance.png'), dpi=300, bbox_inches='tight')
plt.show()

print("Feature importance plot saved")
print(f"\nTop 10 features:")
for i in range(min(10, len(importances))):
    idx = indices[i]
    print(f"  {i+1:2d}. {band_names[idx]:12s}: {importances[idx]:.4f}")

## 10. Save Trained Model

In [ ]:
import joblib
import shutil

# save model and scaler to run folder
model_file = os.path.join(RUN_FOLDER, 'random_forest_model.pkl')
joblib.dump(rf, model_file)

scaler_file = os.path.join(RUN_FOLDER, 'feature_scaler.pkl')
joblib.dump(scaler, scaler_file)

# copy to main models folder for the predictor app
main_model_file = os.path.join(MODELS_PATH, 'random_forest_model.pkl')
main_scaler_file = os.path.join(MODELS_PATH, 'feature_scaler.pkl')
shutil.copy(model_file, main_model_file)
shutil.copy(scaler_file, main_scaler_file)

print(f"Model saved to: {model_file}")
print(f"Scaler saved to: {scaler_file}")
print(f"\nCopied to models folder:")
print(f"  {main_model_file}")
print(f"  {main_scaler_file}")
print(f"\nRemember to apply scaler.transform(X) before prediction.")

## 11. Per-City Performance Analysis (Optional)
### Evaluate model performance on each city individually

In [ ]:
print("\n" + "=" * 70)
print("PER-CITY PERFORMANCE ANALYSIS")
print("=" * 70)

per_city_results = []

for city_data in complete_cities:
    city_name = city_data["name"]
    stack_file = city_data["stack_file"]
    worldcover_file = city_data["worldcover_file"]
    
    print(f"\n{city_name}:")
    
    try:
        with rasterio.open(stack_file) as src:
            X_stack = src.read()
            stack_transform = src.transform
            stack_shape = (src.height, src.width)
            stack_crs = src.crs
        
        with rasterio.open(worldcover_file) as src:
            worldcover_data = np.empty(stack_shape, dtype=np.uint8)
            reproject(
                source=rasterio.band(src, 1),
                destination=worldcover_data,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=stack_transform,
                dst_crs=stack_crs,
                resampling=Resampling.nearest
            )
        
        labels = np.isin(worldcover_data, GREEN_CLASSES).astype(np.uint8)
        
        X = X_stack.reshape(X_stack.shape[0], -1).T
        y = labels.flatten()
        valid_mask = ~np.isnan(X).any(axis=1)
        X_city = X[valid_mask]
        y_city = y[valid_mask]
        
        X_city_scaled = scaler.transform(X_city)
        y_pred_city = rf.predict(X_city_scaled)
        
        acc = accuracy_score(y_city, y_pred_city)
        prec = precision_score(y_city, y_pred_city, zero_division=0)
        rec = recall_score(y_city, y_pred_city, zero_division=0)
        f1_city = f1_score(y_city, y_pred_city, zero_division=0)
        
        gt_green_pct = 100 * np.sum(y_city == 1) / len(y_city)
        pred_green_pct = 100 * np.sum(y_pred_city == 1) / len(y_pred_city)
        diff_pct = pred_green_pct - gt_green_pct
        
        print(f"  Accuracy:  {acc:.4f}")
        print(f"  Precision: {prec:.4f}")
        print(f"  Recall:    {rec:.4f}")
        print(f"  F1-Score:  {f1_city:.4f}")
        print(f"  Green %:   GT={gt_green_pct:.1f}%  Pred={pred_green_pct:.1f}%  (diff: {diff_pct:+.1f}%)")
        
        per_city_results.append({
            "city": city_name,
            "accuracy": float(acc),
            "precision": float(prec),
            "recall": float(rec),
            "f1_score": float(f1_city),
            "gt_green_pct": float(gt_green_pct),
            "pred_green_pct": float(pred_green_pct),
            "diff_pct": float(diff_pct)
        })
        
    except Exception as e:
        print(f"  Error: {e}")

with open(os.path.join(RUN_FOLDER, "per_city_metrics.json"), "w") as f:
    json.dump(per_city_results, f, indent=2)

print(f"\n{'=' * 70}")
print(f"Per-city metrics saved")

## 12. Summary Report

In [ ]:
print("\n" + "=" * 80)
print("MULTI-CITY TRAINING - SUMMARY REPORT")
print("=" * 80)

print(f"\nGround Truth: WorldCover 2021")
print(f"Green Classes: Tree cover (10), Shrubland (20), Grassland (30), Mangroves (95)")

print(f"\nImprovements applied:")
print(f"  1. Balanced sampling (equal green/non-green per city)")
print(f"  2. Feature normalization (StandardScaler)")
print(f"  3. Increased n_estimators: 100 -> 200")
print(f"  4. Removed max_depth limit: 25 -> None")
print(f"  5. Reduced min_samples_split: 50 -> 10")
print(f"  6. Reduced min_samples_leaf: 20 -> 5")

print(f"\nTraining Data:")
print(f"  Cities: {len(complete_cities)}")
for city in complete_cities:
    print(f"    - {city['name']}")

print(f"\n  Training samples: {len(X_train):,}")
print(f"  Testing samples:  {len(X_test):,}")

print(f"\nOverall performance:")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")

if per_city_results:
    print(f"\nPer-city average:")
    avg_acc = np.mean([r['accuracy'] for r in per_city_results])
    avg_prec = np.mean([r['precision'] for r in per_city_results])
    avg_rec = np.mean([r['recall'] for r in per_city_results])
    avg_f1 = np.mean([r['f1_score'] for r in per_city_results])
    avg_diff = np.mean([r['diff_pct'] for r in per_city_results])
    print(f"  Accuracy:  {avg_acc:.4f}")
    print(f"  Precision: {avg_prec:.4f}")
    print(f"  Recall:    {avg_rec:.4f}")
    print(f"  F1-Score:  {avg_f1:.4f}")
    print(f"  Avg green % difference: {avg_diff:+.2f}%")

print(f"\nOutput files in: {RUN_FOLDER}")
print(f"  - metrics.json")
print(f"  - per_city_metrics.json")
print(f"  - confusion_matrix.png")
print(f"  - feature_importance.png")
print(f"  - random_forest_model.pkl")
print(f"  - feature_scaler.pkl")

print(f"\nNote: always apply scaler.transform(X) before prediction.")

print(f"\n" + "=" * 80)
print(f"TRAINING COMPLETE")
print(f"=" * 80)